In [21]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
from scipy.stats import lognorm

import pandas as pd
sys.path.remove(sys.path[0])

sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')
#sys.path.append('/Users/ajc/CODES/ats/tools/utils/')
import get_transect_data

sys.path.append('/Users/ajc/Core/PyScript')
import readdata

In [22]:
indata=h5py.File('/Users/ajc/HOME/data/forcing/Barrow_Alaska/barrow1985-2015/barrow1985-2015-trend-smooth-30pc.h5','r')
outdata=h5py.File('/Users/ajc/HOME/data/forcing/Barrow_Alaska/barrow1985-2015/barrow2015_rain_advectionA.h5','w')

In [23]:
indata.keys()
0.5*9.8*1000 + 101325

106225.0

In [24]:
to_mm_day = 1000*86400
rain = indata['precipitation rain [m s^-1]'][-365:]*to_mm_day
pos = np.where(rain == np.max(rain))[0][0]
print (pos, rain[pos], rain[pos])

temp = indata['air temperature [K]'][-365:]

236 15.206944302000002 15.206944302000002


In [26]:
rain_x1 = np.array(rain)
rain_x2 = np.array(rain)
rain_x3 = np.array(rain)
rain_x2[pos] = rain[pos]*2
rain_x3[pos] = rain[pos]*3

plt.plot(rain_x1,'r')
#plt.plot(rain_x2,'b.')
#plt.plot(rain_x3,'g.')
print (np.mean(rain_x1)*365, np.mean(rain_x2)*365, np.mean(rain_x3)*365)

93.981427453755 109.188371755755 124.395316057755


In [6]:
rain_up_x1 = np.concatenate((rain_x1, rain_x1))
rain_up_x2 = np.concatenate((rain_x1, rain_x2))
rain_up_x3 = np.concatenate((rain_x1, rain_x3))

In [7]:
rain_up_x1 = np.reshape(rain_up_x1,(-1,1))
rain_up_x2 = np.reshape(rain_up_x2,(-1,1))
rain_up_x3 = np.reshape(rain_up_x3,(-1,1))

In [16]:
%matplotlib qt
plt.plot(temp,'g-')
plt.axvline(236)
plt.axhline(273)
plt.axvline(260)
#plt.axhline(277)

temp1 = np.array(temp)
for i,t in enumerate(temp1[237:248]):
    #print (i)
    if i < 7:
        temp1[237+i] = temp1[237] + (temp1[237] - t)
    else:
        temp1[237+i] = (temp1[237+i-1] + temp1[237+i] )/2.
plt.plot(temp1,'r-')
#plt.xlim(200,280)
#for key in indata.keys():
#    print (key)
#    indata.copy(key, outdata)
#outdata.close()
#indata.close()

In [59]:
outfile='/Users/ajc/Documents/MyPapers/RainWarming/figures/'
fig, axs = plt.subplots(1,2, figsize=(6,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
axs = axs.ravel()
axs[0].plot(rain_x1,color='k',linestyle='solid',linewidth=1.)
axs[0].set_xlabel('Time [day]',fontsize=12, fontweight='normal')
axs[0].set_ylabel('Precipitation rain [mm]',fontsize=12, fontweight='normal')
axs[0].set_xlim(130,260)
axs[0].set_ylim(0,20)

temp_t = [t - 273.15 for t in temp]
axs[1].plot(temp_t,color='k',linestyle='solid',linewidth=1.)
axs[1].set_xlabel('Time [day]',fontsize=12, fontweight='normal')
axs[1].set_ylabel('Air temperature [\u00B0C]',fontsize=12, fontweight='normal')
axs[1].set_xlim(130,260)
axs[1].set_ylim(-5,15)
#axs[0].axvline(236,ymin=0.0, ymax=1.7,color='lightgray',linestyle='dashed',linewidth=2)
axs[1].axvline(236,ymin=0.0, ymax=1.7,color='lightgray',linestyle='dashed',linewidth=1)
plt.tight_layout()
plt.savefig(outfile+'rain_airT.jpeg', bbox_inches='tight', dpi=300)
plt.show()

In [10]:
temp_up = np.concatenate((temp, temp1))
temp_up = np.reshape(temp_up,(-1,1))

In [11]:
time = np.arange(0.5,730.5,1)*86400
time[:2]
print (np.max(rain_up_x1), rain_up_x1[365+236],(365+236+7)*86400)



1.7600629979166669e-07 [1.760063e-07] 52531200


In [12]:
for key in indata.keys():
    if 'rain' in key:
        print ('R', key+', x1.5')
        outdata.create_dataset(key + ', 1x' , data=rain_up_x1)
        outdata.create_dataset(key + ', 2x' , data=rain_up_x2)
        outdata.create_dataset(key + ', 3x' , data=rain_up_x3)
    if 'time' in key:
        print ('T')
        outdata.create_dataset(key, data=time)
    elif 'rain' not in key and 'time' not in key:
        print (key)
        d = np.concatenate((indata[key][-365:], indata[key][-365:]))
        outdata.create_dataset(key, data=d)

outdata.create_dataset('air temperature [K], increased', data=temp_up)
outdata.close()

air temperature [K]
incoming longwave radiation [W m^-2]
incoming shortwave radiation [W m^-2]
R precipitation rain [m s^-1], x1.5
precipitation snow [m SWE s^-1]
relative humidity [-]
T
wind speed [m s^-1]


In [ ]:
#mass flux [moles/m/s]
# 20 mm/day

#300 cm3/s
side = 17 #m
q = -1
to_cc = 1e6
mrho= 1000/0.018 #[moles/m3]
mass = q * side / mrho # moles/m * 1/s * m * m3/moles = m3/s
print (mass)# * to_cc)

In [ ]:
#1 cc = 1000 mm3
mass = 20 #mm/day
mass_m = mass * 0.001
2/864

In [ ]:
2.315 * 1e-6 * mrho

In [ ]:
factor = 1.16 * 1e-8 
src = 20 *factor * mrho
src